<h1> This notebook is to create make a dataframe of having the list of Toronto's Boroughs with their zipcodes attached to them </h1>

Lets first get the particular page of wikipedia having the said values

In [1]:
import lxml
from lxml import html
import requests
import pandas as pd

Let's first import the beautiful Soup library and the urlLib library

In [2]:
from urllib.request import urlopen

from bs4 import BeautifulSoup as BS

Now we open the url page in an object and then extract the table using findall to an object named table

In [3]:
wikiPage = urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M') 

In [4]:
soup = BS(wikiPage,'lxml')

In [5]:
table = soup.find_all('tbody')[0]

now let's create a dataframe to save the values of everything required

In [6]:
dataTable = pd.DataFrame(columns=range(0,3), index = [0])

Let's add the column names now & insert the values

In [7]:
dataTable.columns = ['PostCode','Borough','Neighbourhood']

In [8]:
rowCounter = 0
for row in table.find_all('tr'):
    columnValue = row.find_all('td')
    columncounter =0
    for column in columnValue :
        if(column.get_text() != 'Not assigned'):
            if(columncounter == 0):
                dataTable.set_value(rowCounter,'PostCode',column.get_text())
            elif(columncounter == 1):
                dataTable.set_value(rowCounter,'Borough',column.get_text())
            elif(columncounter == 2):
                dataTable.set_value(rowCounter,'Neighbourhood',column.get_text())
        columncounter = columncounter + 1
    rowCounter = rowCounter + 1
    

C:\Users\ezchave\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
C:\Users\ezchave\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if sys.path[0] == '':
C:\Users\ezchave\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # Remove the CWD from sys.path while we load stuff.


let's see the result

In [9]:
dataTable.head()

,PostCode,Borough,Neighbourhood
0,NaN,NaN,NaN
1,M1A,NaN,Not assigned\n
2,M2A,NaN,Not assigned\n
3,M3A,North York,Parkwoods\n
4,M4A,North York,Victoria Village\n


Now we need to remove the NaN values, not assigned value's rows and the index needs tobe reshuffled accordignly

In [10]:
dataTable.dropna(inplace = True)

In [11]:
dataTable = dataTable[dataTable.Borough != 'Not assigned']


In [12]:
dataTable.reset_index(inplace = True)

In [13]:
dataTable.head()

,index,PostCode,Borough,Neighbourhood
0,3,M3A,North York,Parkwoods\n
1,4,M4A,North York,Victoria Village\n
2,5,M5A,Downtown Toronto,Harbourfront\n
3,6,M5A,Downtown Toronto,Regent Park\n
4,7,M6A,North York,Lawrence Heights\n


In [14]:
dataTable = dataTable.drop(['index'],axis = 1)

Now we remove the \n from the name fo neighbourhoods, group the common neighbourhoods & boroughs according to the postCodes

In [15]:
dataTable['Neighbourhood'] = dataTable['Neighbourhood'].map(lambda x : x.rstrip('\n'))

In [16]:
dataTable['Borough'] = dataTable[['PostCode','Borough','Neighbourhood']].groupby(['PostCode'])['Borough'].transform(lambda x : ','.join(x))

In [17]:
dataTable['Neighbourhood'] = dataTable[['PostCode','Borough','Neighbourhood']].groupby(['PostCode'])['Neighbourhood'].transform(lambda x : ','.join(x))

In [18]:
dataTable.drop_duplicates(inplace = True)

In [19]:
dataTable.reset_index(inplace = True)

In [20]:
dataTable = dataTable.drop(['index'],axis = 1)

Let's see how it looks now

In [21]:
dataTable.head()

,PostCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,"Downtown Toronto,Downtown Toronto","Harbourfront,Regent Park"
3,M6A,"North York,North York","Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Not assigned


Let's import the geocoder module to see the values of Lattitude's & Longitudes

In [22]:
from geopy import geocoders

# USing the geo code data instead of geoCoder as for some reason it's not working

In [23]:
geoData = pd.read_csv('https://cocl.us/Geospatial_data')

In [24]:
geoData.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Let's rename the column Psotal Codes to PostCode

In [25]:
geoData['PostCode'] = geoData['Postal Code']

In [26]:
geoData = geoData.drop('Postal Code', axis = 1)

Now we merge the geoData table and the dataTable to get the values in one table

In [27]:
dataTable = pd.merge(dataTable,geoData,how = 'inner', on = 'PostCode')

Let's see how it looks now

In [28]:
dataTable.head()

,PostCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,"Downtown Toronto,Downtown Toronto","Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,"North York,North York","Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Not assigned,43.662301,-79.389494


As mentioned earlier , the final cell of the notebook is the shape of the table

In [29]:
dataTable.shape

(103, 5)

In [30]:
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


Let's see how many Boroughs and neighbourhoods are there in our data

In [31]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(dataTable['Borough'].unique()),
        dataTable.shape[0]
    )
)

The dataframe has 32 boroughs and 103 neighborhoods.


Let's get the lat and long of Toronto

In [32]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


Let's create the map

In [33]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(dataTable['Latitude'], dataTable['Longitude'], dataTable['Borough'], dataTable['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [34]:
toronto_data = pd.DataFrame(columns = ['lat','long','borough','Neighbourhood'],index = [0])

In [35]:
indx = 0
for lat,long,borough,neigh,index in zip(dataTable['Latitude'], dataTable['Longitude'], dataTable['Borough'], dataTable['Neighbourhood'],dataTable.index):
    if('Toronto' in borough):
        toronto_data.set_value(indx,'lat',lat)
        toronto_data.set_value(indx,'long',long)
        toronto_data.set_value(indx,'borough',borough)
        toronto_data.set_value(indx,'Neighbourhood',neigh)
        indx = indx + 1


C:\Users\ezchave\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  after removing the cwd from sys.path.
C:\Users\ezchave\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  """
C:\Users\ezchave\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
C:\Users\ezchave\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys


In [36]:
toronto_data.head()

,lat,long,borough,Neighbourhood
0,43.6543,-79.3606,"Downtown Toronto,Downtown Toronto","Harbourfront,Regent Park"
1,43.6572,-79.3789,"Downtown Toronto,Downtown Toronto","Ryerson,Garden District"
2,43.6515,-79.3754,Downtown Toronto,St. James Town
3,43.6764,-79.293,East Toronto,The Beaches
4,43.6448,-79.3733,Downtown Toronto,Berczy Park


Now let's use foresquare for the mapping 

In [37]:
CLIENT_ID = 'HQADI3DUD22CJBEMCEI4MPC4PXS3SVYOJH1DYGOKQIEBPDIQ' # your Foursquare ID
CLIENT_SECRET = 'BJ4DBMDCHV40OW451L4OFEBOLX4FFNJTJHNYIMLZ2J3ZYHYV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HQADI3DUD22CJBEMCEI4MPC4PXS3SVYOJH1DYGOKQIEBPDIQ
CLIENT_SECRET:BJ4DBMDCHV40OW451L4OFEBOLX4FFNJTJHNYIMLZ2J3ZYHYV


In [38]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        LIMIT = 500
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [39]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['lat'],
                                   longitudes=toronto_data['long']
                                  )
    

Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide,King,Richmond
Dovercourt Village,Dufferin
Harbourfront East,Toronto Islands,Union Station
Little Portugal,Trinity
The Danforth West,Riverdale
Design Exchange,Toronto Dominion Centre
Brockton,Exhibition Place,Parkdale Village
The Beaches West,India Bazaar
Commerce Court,Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North,Forest Hill West
High Park,The Junction South
North Toronto West
The Annex,North Midtown,Yorkville
Parkdale,Roncesvalles
Davisville
Harbord,University of Toronto
Runnymede,Swansea
Moore Park,Summerhill East
Chinatown,Grange Park,Kensington Market
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown,St. James Town
First Canadian Place,Underground city


In [40]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,57,57,57,57,57,57
"Brockton,Exhibition Place,Parkdale Village",19,19,19,19,19,19
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",15,15,15,15,15,15
"Cabbagetown,St. James Town",44,44,44,44,44,44
Central Bay Street,88,88,88,88,88,88
"Chinatown,Grange Park,Kensington Market",100,100,100,100,100,100
Christie,16,16,16,16,16,16


In [41]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 236 uniques categories.


now we analyze each nieghbourhood

In [42]:
# one hot encoding
toronoto_ohe = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronoto_ohe['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronoto_ohe.columns[-1]] + list(toronoto_ohe.columns[:-1])
toronoto_ohe = toronoto_ohe[fixed_columns]

toronoto_ohe.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
toronto_grouped = toronoto_ohe.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide,King,Richmond",0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.010000,0.0,0.0,0.01,0.0
1,Berczy Park,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.017544,0.0,0.0,0.00,0.0
2,"Brockton,Exhibition Place,Parkdale Village",0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0
3,Business Reply Mail Processing Centre 969 Eastern,0.058824,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.000000,0.0,0.0,0.066667,0.066667,0.066667,0.133333,0.2,0.133333,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0


#### Let's print each neighborhood along with the top 5 most common venues

In [51]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    '''temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')'''

----Adelaide,King,Richmond----
----Berczy Park----
----Brockton,Exhibition Place,Parkdale Village----
----Business Reply Mail Processing Centre 969 Eastern----
----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
----Cabbagetown,St. James Town----
----Central Bay Street----
----Chinatown,Grange Park,Kensington Market----
----Christie----
----Church and Wellesley----
----Commerce Court,Victoria Hotel----
----Davisville----
----Davisville North----
----Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West----
----Design Exchange,Toronto Dominion Centre----
----Dovercourt Village,Dufferin----
----First Canadian Place,Underground city----
----Forest Hill North,Forest Hill West----
----Harbord,University of Toronto----
----Harbourfront East,Toronto Islands,Union Station----
----Harbourfront,Regent Park----
----High Park,The Junction South----
----Lawrence Park----
----Little Portugal,Trinity----
----Moore Park,Summerhill East--

In [52]:
temp

,index,37
0,Neighborhood,"The Danforth West,Riverdale"
1,Yoga Studio,0.0227273
2,Adult Boutique,0
3,Afghan Restaurant,0
4,Airport,0
5,Airport Food Court,0
6,Airport Gate,0
7,Airport Lounge,0
8,Airport Service,0
9,Airport Terminal,0


In [45]:
import numpy as np

In [46]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [47]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Thai Restaurant,Steakhouse,American Restaurant,Bar,Burger Joint,Gym,Hotel,Bakery
1,Berczy Park,Coffee Shop,Cocktail Bar,Café,Cheese Shop,Farmers Market,Beer Bar,Steakhouse,Seafood Restaurant,Restaurant,Bakery
2,"Brockton,Exhibition Place,Parkdale Village",Café,Breakfast Spot,Coffee Shop,Grocery Store,Italian Restaurant,Caribbean Restaurant,Stadium,Bar,Furniture / Home Store,Burrito Place
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Garden,Recording Studio,Auto Workshop,Skate Park,Burrito Place,Fast Food Restaurant,Farmers Market,Brewery,Restaurant
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Sculpture Garden,Boutique,Plane,Boat or Ferry,Airport Gate,Airport


In [64]:
toronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide,King,Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.00,0.00000,0.000000,0.00,0.010000,0.000000,0.000000,0.010000,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.00,0.00000,0.000000,0.00,0.017544,0.000000,0.000000,0.000000,0.000000
2,"Brockton,Exhibition Place,Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.00,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.00,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.000000,0.000000,0.000000,0.066667,0.066667,0.066667,0.133333,0.2,0.133333,...,0.000000,0.00,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Cabbagetown,St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.00,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
6,Central Bay Street,0.011364,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.00,0.00000,0.000000,0.00,0.011364,0.000000,0.000000,0.011364,0.000000
7,"Chinatown,Grange Park,Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.01,0.01000,0.000000,0.00,0.060000,0.000000,0.030000,0.010000,0.000000
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.00,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
9,Church and Wellesley,0.011364,0.011364,0.011364,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.011364,0.00,0.00000,0.000000,0.00,0.000000,0.011364,0.011364,0.000000,0.011364


# now let's make the clusters

In [48]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [61]:
toronto_data.rename(columns = {'Neighbourhood' : 'Neighborhood'},inplace = True)

In [62]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')


In [63]:

toronto_merged 


,lat,long,borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,43.6543,-79.3606,"Downtown Toronto,Downtown Toronto","Harbourfront,Regent Park",1,Coffee Shop,Park,Café,Pub,Bakery,Mexican Restaurant,Breakfast Spot,Theater,Chocolate Shop,Beer Store
1,43.6572,-79.3789,"Downtown Toronto,Downtown Toronto","Ryerson,Garden District",1,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Tea Room,Pizza Place,Sporting Goods Shop,Japanese Restaurant,Fast Food Restaurant
2,43.6515,-79.3754,Downtown Toronto,St. James Town,1,Coffee Shop,Hotel,Restaurant,Café,Cosmetics Shop,Breakfast Spot,Gastropub,Bakery,Italian Restaurant,Diner
3,43.6764,-79.293,East Toronto,The Beaches,1,Health Food Store,Pub,Music Venue,Wings Joint,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
4,43.6448,-79.3733,Downtown Toronto,Berczy Park,1,Coffee Shop,Cocktail Bar,Café,Cheese Shop,Farmers Market,Beer Bar,Steakhouse,Seafood Restaurant,Restaurant,Bakery
5,43.658,-79.3874,Downtown Toronto,Central Bay Street,1,Coffee Shop,Café,Italian Restaurant,Burger Joint,Chinese Restaurant,Japanese Restaurant,Restaurant,Sandwich Place,Bubble Tea Shop,Bar
6,43.6695,-79.4226,Downtown Toronto,Christie,1,Grocery Store,Café,Park,Restaurant,Nightclub,Athletics & Sports,Baby Store,Diner,Italian Restaurant,Convenience Store
7,43.6506,-79.3846,"Downtown Toronto,Downtown Toronto,Downtown Tor...","Adelaide,King,Richmond",1,Coffee Shop,Café,Thai Restaurant,Steakhouse,American Restaurant,Bar,Burger Joint,Gym,Hotel,Bakery
8,43.669,-79.4423,"West Toronto,West Toronto","Dovercourt Village,Dufferin",1,Supermarket,Bakery,Pharmacy,Music Venue,Middle Eastern Restaurant,Bank,Bar,Discount Store,Pool,Café
9,43.6408,-79.3818,"Downtown Toronto,Downtown Toronto,Downtown Tor...","Harbourfront East,Toronto Islands,Union Station",1,Coffee Shop,Hotel,Aquarium,Italian Restaurant,Café,Fried Chicken Joint,Restaurant,Scenic Lookout,Pizza Place,Bakery


In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['lat'], toronto_merged['long'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters